This is an example

In [1]:
sFilename_nhd = '6'
print(sFilename_nhd)

6


Now set up the case object. We provide three options to set up a simulation case. First, from scratch (iFlag_option: 1); second, using a json file (iFlag_option = 2). For details of the json file format, please read the documentation.

In [ ]:
from pyflowline.case.pycase import flowlinecase
from pyflowline.case.pyflowline_read_model_configuration_file import pyflowline_read_model_configuration_file

iFlag_option = 2

if iFlag_option ==1:

    oPyflowline=flowlinecase()
    oPyflowline.iCase_index = 1


else: 
    if iFlag_option ==2:
        sFilename_configuration_in = '/qfs/people/liao313/workspace/python/pyflowline/pyflowline/config/pyflowline_susquehanna_hexagon.json'
        oPyflowline = pyflowline_read_model_configuration_file(sFilename_configuration_in)
        

        



After the case object was created, we can run the mainly three types of operations. 
First, we can preprocess the flowline.

Before the operation, we can visualize the original or raw flowline dataset. For most visualization, all you need to provide is a keyword, a look up table is provided in the documentation.

In [ ]:
from pyflowline.plot.pyflowline_plot_flowline import pyflowline_plot_flowline
pyflowline_plot_flowline(oPyflowline, sVariable_in = 'flowline_raw')

In [ ]:
from pyflowline.operation.preprocess_flowline_op import preprocess_flowline_op
preprocess_flowline_op(oPyflowline)

After the preprocess, we can visualize the final flowline 

In [ ]:
pyflowline_plot_flowline(oPyflowline, sVariable_in = 'flowline_simplified')

Second, we can creata a mesh

In [ ]:
from pyflowline.operation.create_mesh_op import create_mesh_op
aCell = create_mesh_op(oPyflowline)

We can also visualize the mesh:

In [ ]:
from pyflowline.plot.pyflowline_plot_mesh import pyflowline_plot_mesh
pyflowline_plot_mesh(oPyflowline)

Last, we can operate the intersect to generate desired flowlines. But we recommend that we run the intersect with some postprocess.

In [ ]:
from pyflowline.operation.intersect_flowline_with_mesh_with_postprocess_op import intersect_flowline_with_mesh_with_postprocess_op
intersect_flowline_with_mesh_with_postprocess_op(oPyflowline)

After this step, we can now visualize the final flowline again.

In [ ]:
pyflowline_plot_flowline(oPyflowline, sVariable_in = 'flowline_final')